## 1. Install libraries and load data

#### Install libraries for the project

In [1]:
# Install libraries

!pip install geopy

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Preparation - load 3 datasets and pre-processing: 
#### San Francisco zipcode and neighborhood, 
http://www.healthysf.org/bdi/outcomes/zipmap.htm
#### Zipcode and coordinate database
https://www.unitedstateszipcodes.org/zip-code-database/
#### Population in San Francisco by zipcode
http://zipatlas.com/us/ca/san-francisco/zip-code-comparison/percentage-chinese-population.htm


In [2]:
SF_zipcode = pd.read_csv("SF zipcode.csv")
SF_zipcode.head()

,Zip Code,Neighborhood
0,94102,Hayes Valley/Tenderloin/North of Market
1,94103,South of Market
2,94107,Potrero Hill
3,94108,Chinatown
4,94109,Polk/Russian Hill (Nob Hill)


In [3]:
Zipcode=pd.read_csv("zip_code_database.csv")
Zipcode.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,501,UNIQUE,0,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0


In [7]:
SF_population=pd.read_csv("Chinese percentage of SF.csv")
SF_population.head()

,Unnamed: 0,Zip Code,Location,City,Population,% Chinese,National Rank
0,1,94104,"37.791222, -122.402241","San Francisco, California",374,58.28%,#1
1,2,94133,"37.802071, -122.411004","San Francisco, California","26,827",51.31%,#3
2,3,94108,"37.791998, -122.408653","San Francisco, California","13,716",50.75%,#4
3,4,94116,"37.744410, -122.486764","San Francisco, California","42,958",39.50%,#10
4,5,94122,"37.760412, -122.484966","San Francisco, California","55,492",34.10%,#13


In [5]:
# Preprocessing Zipcode database 
# Select only 'zip', 'latitude' and 'longtitude', create a new dataframe 
latlng=Zipcode[['zip','latitude','longitude']]
latlng.head()

,zip,latitude,longitude
0,501,40.81,-73.04
1,544,40.81,-73.04
2,601,18.16,-66.72
3,602,18.38,-67.18
4,603,18.43,-67.15


In [6]:
# Merge 'SF_zipcode' dataset and 'latlng' dataset by zip code
df_SFdata = SF_zipcode.merge(latlng, left_on="Zip Code", right_on="zip", how="left")

# Remove 'zip' column
df_SFdata = df_SFdata.drop('zip', 1)

df_SFdata.head()

,Zip Code,Neighborhood,latitude,longitude
0,94102,Hayes Valley/Tenderloin/North of Market,37.78,-122.42
1,94103,South of Market,37.77,-122.41
2,94107,Potrero Hill,37.76,-122.39
3,94108,Chinatown,37.79,-122.41
4,94109,Polk/Russian Hill (Nob Hill),37.79,-122.42


In [8]:
# Pre-processing 'SF_population' dataset: same way as with Zipcode database
# Select zipcode and population as a new dataframe, merge with 'df_SFdata'

pop_zip=SF_population[['Zip Code','Population']]
df_SFdata = df_SFdata.merge(pop_zip, on="Zip Code", how="left")

df_SFdata

,Zip Code,Neighborhood,latitude,longitude,Population
0,94102,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,"28,991"
1,94103,South of Market,37.77,-122.41,"23,016"
2,94107,Potrero Hill,37.76,-122.39,"17,368"
3,94108,Chinatown,37.79,-122.41,"13,716"
4,94109,Polk/Russian Hill (Nob Hill),37.79,-122.42,"56,322"
5,94110,Inner Mission/Bernal Heights,37.74,-122.41,"74,633"
6,94112,Ingelside-Excelsior/Crocker-Amazon,37.72,-122.44,"73,104"
7,94114,Castro/Noe Valley,37.75,-122.43,"30,574"
8,94115,Western Addition/Japantown,37.78,-122.43,"33,115"
9,94116,Parkside/Forest Hill,37.74,-122.48,"42,958"


Data loading and preparation is completed
A dataframe with zipcode, neighborhood, coordinates, population and Chinese percentage has been formed

## 2. Create a map of San Francisco and use FourSquare to get venues of all neighborhoods
#### Create a map of SF with neighborhoods superimposed on top.

In [9]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="SF_explorer")
location = geolocator.geocode("Union Square, San Francisco")
print(location.address)
print('The geograpical coordinate of San Francisco, CA are {}, {}.'.format(location.latitude, location.longitude))

Union Square, San Francisco, San Francisco City and County, California, United States of America
The geograpical coordinate of San Francisco, CA are 37.7879363, -122.40751740318035.


In [10]:
# create map of San Francisco using latitude and longitude values
map_sf = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_SFdata['latitude'], df_SFdata['longitude'], df_SFdata['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sf)  
    
map_sf

#### Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = 'K24VV2FJTUBULJ3N0PVBMNHY5GQD3WUO0ELJA0WN0L1NWBK5' # your Foursquare ID
CLIENT_SECRET = 'DTLBHE5ZL2VXBYEQV00UYTAAN01AUIZ4ADDGEWQL1RZ5HRS4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K24VV2FJTUBULJ3N0PVBMNHY5GQD3WUO0ELJA0WN0L1NWBK5
CLIENT_SECRET:DTLBHE5ZL2VXBYEQV00UYTAAN01AUIZ4ADDGEWQL1RZ5HRS4


Create the GET request URL

In [12]:
LIMIT=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, df_SFdata["latitude"], df_SFdata["longitude"], VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=K24VV2FJTUBULJ3N0PVBMNHY5GQD3WUO0ELJA0WN0L1NWBK5&client_secret=DTLBHE5ZL2VXBYEQV00UYTAAN01AUIZ4ADDGEWQL1RZ5HRS4&ll=0     37.78\n1     37.77\n2     37.76\n3     37.79\n4     37.79\n5     37.74\n6     37.72\n7     37.75\n8     37.78\n9     37.74\n10    37.77\n11    37.78\n12    37.77\n13    37.76\n14    37.80\n15    37.73\n16    37.74\n17    37.74\n18    37.72\n19    37.80\n20    37.72\nName: latitude, dtype: float64,0    -122.42\n1    -122.41\n2    -122.39\n3    -122.41\n4    -122.42\n5    -122.41\n6    -122.44\n7    -122.43\n8    -122.43\n9    -122.48\n10   -122.45\n11   -122.46\n12   -122.49\n13   -122.48\n14   -122.43\n15   -122.38\n16   -122.46\n17   -122.44\n18   -122.48\n19   -122.41\n20   -122.41\nName: longitude, dtype: float64&v=20180605&radius=500&limit=100'

#### Create a function to get veneus of different zipcode with radius of 500m

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
SF_venues = getNearbyVenues(names=df_SFdata['Neighborhood'],
                                   latitudes=df_SFdata['latitude'],
                                   longitudes=df_SFdata['longitude']
                                  )

Hayes Valley/Tenderloin/North of Market
South of Market
Potrero Hill
Chinatown
Polk/Russian Hill (Nob Hill)
Inner Mission/Bernal Heights
Ingelside-Excelsior/Crocker-Amazon
Castro/Noe Valley
Western Addition/Japantown
Parkside/Forest Hill
Haight-Ashbury
Inner Richmond
Outer Richmond
Sunset
Marina
Bayview-Hunters Point
St. Francis Wood/Miraloma/West Portal
Twin Peaks-Glen Park
Lake Merced
North Beach/Chinatown
Visitacion Valley/Sunnydale


In [15]:
# Check the size of the resulting dataframe
print(SF_venues.shape)
SF_venues.head()

(1069, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Herbst Theater,37.779548,-122.420953,Concert Hall
1,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,War Memorial Opera House,37.778601,-122.420816,Opera House
2,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,San Francisco Ballet,37.778580,-122.420798,Dance Studio
3,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Louise M. Davies Symphony Hall,37.777976,-122.420157,Concert Hall
4,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,War Memorial Court,37.779042,-122.420971,Park


In [16]:
# Check how many venues were returned for each neighborhood

SF_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bayview-Hunters Point,4,4,4,4,4,4
Castro/Noe Valley,83,83,83,83,83,83
Chinatown,86,86,86,86,86,86
Haight-Ashbury,83,83,83,83,83,83
Hayes Valley/Tenderloin/North of Market,99,99,99,99,99,99
Ingelside-Excelsior/Crocker-Amazon,43,43,43,43,43,43
Inner Mission/Bernal Heights,35,35,35,35,35,35
Inner Richmond,62,62,62,62,62,62
Lake Merced,21,21,21,21,21,21


## 3. Analyze Each Neighborhood

In [17]:
# one hot encoding
SF_onehot = pd.get_dummies(SF_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
SF_onehot['Neighborhood'] = SF_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [SF_onehot.columns[-1]] + list(SF_onehot.columns[:-1])
SF_onehot = SF_onehot[fixed_columns]

SF_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Bike Shop,Board Shop,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Mountain,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Restaurant,Rock Club,Sake Bar,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Sicilian Restaurant,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Trattoria/Osteria,Tree,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [18]:
# View the size of dataset
SF_onehot.shape

(1069, 234)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [19]:
SF_grouped = SF_onehot.groupby('Neighborhood').mean().reset_index()
SF_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Bike Shop,Board Shop,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Mountain,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Restaurant,Rock Club,Sake Bar,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Sicilian Restaurant,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Trattoria/Osteria,Tree,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,Bayview-Hunters Point,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00

Select venues of restaurants to form a new dataframe 'SF_dine'

In [20]:
SF_restaurant=SF_grouped.loc[:, SF_grouped.columns.str.contains('Restaurant')]
SF_bar=SF_grouped.loc[:, SF_grouped.columns.str.contains('Bar')]
SF_food=SF_grouped.loc[:, SF_grouped.columns.str.contains('Food')]

In [26]:
SF_dine=pd.concat([SF_restaurant, SF_bar, SF_food], axis=1)

SF_dine.head()

,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Burmese Restaurant,Caribbean Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Hawaiian Restaurant,Hunan Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,North Indian Restaurant,Persian Restaurant,Peruvian Restaurant,Ramen Restaurant,Restaurant,Salvadoran Restaurant,Seafood Restaurant,Sicilian Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Bar,Beer Bar,Cocktail Bar,Dive Bar,Gay Bar,Hotel Bar,Juice Bar,Karaoke Bar,Sake Bar,Salon / Barbershop,Sports Bar,Tiki Bar,Wine Bar,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Southern / Soul Food Restaurant,Street Food Gathering
0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
1,0.0,0.012048,0.0,0.000000,0.012048,0.000000,0.024096,0.0,0.0,0.0,0.000000,0.000000,0.012048,0.000000,0.0,0.000000,0.012048,0.0,0.012048,0.000000,0.0,0.012048,0.000000,0.048193,0.000000,0.0,0.000000,0.0,0.012048,0.000000,0.012048,0.0,0.0,0.0,0.0,0.000000,0.0,0.036145,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.012048,0.000000,0.000000,0.0,0.000000,0.000000,0.012048,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
2,0.0,0.023256,0.0,0.011628,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.011628,0.000000,0.000000,0.011628,0.0,0.000000,0.023256,0.0,0.000000,0.011628,0.0,0.000000,0.011628,0.011628,0.000000,0.0,0.011628,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.023256,0.0,0.000000,0.000000,0.0,0.000000,0.011628,0.023256,0.011628,0.046512,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.011628,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
3,0.0,0.012048,0.0,0.000000,0.000000,0.012048,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.012048,0.012048,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.012048,0.012048,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.012048,0.012048,0.0,0.012048,0.012048,0.012048,0.000000,0.012048,0.0,0.012048,0.000000,0.000000,0.0,0.0,0.000000,0.012048,0.000000,0.012048,0.0,0.000000,0.0,0.0,0.000000,0.0
4,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.010101,0.0,0.0,0.0,0.000000,0.030303,0.000000,0.000000,0.0,0.010101,0.010101,0.0,0.000000,0.000000,0.0,0.000000,0.010101,0.010101,0.000000,0.0,0.000000,0.0,0.000000,0.010101,0.030303,0.0,0.0,0.0,0.0,0.020202,0.0,0.020202,0.0,0.000000,0.020202,0.0,0.030303,0.040404,0.010101,0.020202,0.020202,0.0,0.000000,0.010101,0.010101,0.0,0.0,0.000000,0.000000,0.010101,0.030303,0.0,0.010101,0.0,0.0,0.020202,0.0


In [27]:
# Add the column 'Neighborhood'
SF_dine['Neighborhood'] = SF_grouped['Neighborhood']

# Move neighborhood column to the first column
fixed_columns_1 = [SF_dine.columns[-1]] + list(SF_dine.columns[:-1])
SF_dine = SF_dine[fixed_columns_1]

In [28]:
SF_dine.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Burmese Restaurant,Caribbean Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Fast Food Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Hawaiian Restaurant,Hunan Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,North Indian Restaurant,Persian Restaurant,Peruvian Restaurant,Ramen Restaurant,Restaurant,Salvadoran Restaurant,Seafood Restaurant,Sicilian Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Bar,Beer Bar,Cocktail Bar,Dive Bar,Gay Bar,Hotel Bar,Juice Bar,Karaoke Bar,Sake Bar,Salon / Barbershop,Sports Bar,Tiki Bar,Wine Bar,Fast Food Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Street Food Gathering
0,Bayview-Hunters Point,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,Castro/Noe Valley,0.0,0.012048,0.0,0.000000,0.012048,0.000000,0.024096,0.0,0.0,0.0,0.0,0.000000,0.000000,0.012048,0.000000,0.0,0.000000,0.012048,0.0,0.012048,0.000000,0.0,0.012048,0.000000,0.048193,0.000000,0.0,0.000000,0.0,0.012048,0.000000,0.012048,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.036145,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.012048,0.000000,0.000000,0.0,0.000000,0.000000,0.012048,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,Chinatown,0.0,0.023256,0.0,0.011628,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.011628,0.000000,0.000000,0.011628,0.0,0.000000,0.023256,0.0,0.000000,0.011628,0.0,0.000000,0.011628,0.011628,0.000000,0.0,0.011628,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.023256,0.0,0.000000,0.000000,0.0,0.000000,0.011628,0.023256,0.011628,0.046512,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.011628,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
3,Haight-Ashbury,0.0,0.012048,0.0,0.000000,0.000000,0.012048,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.012048,0.012048,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.012048,0.012048,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.012048,0.012048,0.0,0.012048,0.012048,0.012048,0.000000,0.012048,0.0,0.012048,0.000000,0.000000,0.0,0.0,0.000000,0.012048,0.000000,0.012048,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
4,Hayes Valley/Tenderloin/North of Market,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.010101,0.0,0.0,0.0,0.0,0.000000,0.030303,0.000000,0.000000,0.0,0.010101,0.010101,0.0,0.000000,0.000000,0.0,0.000000,0.010101,0.010101,0.000000,0.0,0.000000,0.0,0.000000,0.010101,0.030303,0.0,0.0,0.0,0.0,0.020202,0.020202,0.0,0.020202,0.0,0.000000,0.020202,0.0,0.030303,0.040404,0.010101,0.020202,0.020202,0.0,0.000000,0.010101,0.010101,0.0,0.0,0.000000,0.000000,0.010101,0.030303,0.0,0.0,0.010101,0.0,0.0,0.020202,0.020202,0.0


In [29]:
SF_dine_percentage = SF_dine.merge(df_SFdata, on="Neighborhood", how="left")
SF_dine_percentage.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Burmese Restaurant,Caribbean Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Fast Food Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Hawaiian Restaurant,Hunan Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,North Indian Restaurant,Persian Restaurant,Peruvian Restaurant,Ramen Restaurant,Restaurant,Salvadoran Restaurant,Seafood Restaurant,Sicilian Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Bar,Beer Bar,Cocktail Bar,Dive Bar,Gay Bar,Hotel Bar,Juice Bar,Karaoke Bar,Sake Bar,Salon / Barbershop,Sports Bar,Tiki Bar,Wine Bar,Fast Food Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Street Food Gathering,Zip Code,latitude,longitude,Population
0,Bayview-Hunters Point,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,94124,37.73,-122.38,"33,170"
1,Castro/Noe Valley,0.0,0.012048,0.0,0.000000,0.012048,0.000000,0.024096,0.0,0.0,0.0,0.0,0.000000,0.000000,0.012048,0.000000,0.0,0.000000,0.012048,0.0,0.012048,0.000000,0.0,0.012048,0.000000,0.048193,0.000000,0.0,0.000000,0.0,0.012048,0.000000,0.012048,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.036145,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.012048,0.000000,0.000000,0.0,0.000000,0.000000,0.012048,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,94114,37.75,-122.43,"30,574"
2,Chinatown,0.0,0.023256,0.0,0.011628,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.011628,0.000000,0.000000,0.011628,0.0,0.000000,0.023256,0.0,0.000000,0.011628,0.0,0.000000,0.011628,0.011628,0.000000,0.0,0.011628,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.023256,0.0,0.000000,0.000000,0.0,0.000000,0.011628,0.023256,0.011628,0.046512,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.011628,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,94108,37.79,-122.41,"13,716"
3,Haight-Ashbury,0.0,0.012048,0.0,0.000000,0.000000,0.012048,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.012048,0.012048,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.012048,0.012048,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.012048,0.012048,0.0,0.012048,0.012048,0.012048,0.000000,0.012048,0.0,0.012048,0.000000,0.000000,0.0,0.0,0.000000,0.012048,0.000000,0.012048,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,94117,37.77,-122.45,"38,738"
4,Hayes Valley/Tenderloin/North of Market,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.010101,0.0,0.0,0.0,0.0,0.000000,0.030303,0.000000,0.000000,0.0,0.010101,0.010101,0.0,0.000000,0.000000,0.0,0.000000,0.010101,0.010101,0.000000,0.0,0.000000,0.0,0.000000,0.010101,0.030303,0.0,0.0,0.0,0.0,0.020202,0.020202,0.0,0.020202,0.0,0.000000,0.020202,0.0,0.030303,0.040404,0.010101,0.020202,0.020202,0.0,0.000000,0.010101,0.010101,0.0,0.0,0.000000,0.000000,0.010101,0.030303,0.0,0.0,0.010101,0.0,0.0,0.020202,0.020202,0.0,94102,37.78,-122.42,"28,991"


In [30]:
# Change percentage string to number for normalization

SF_dine_percentage['Population']=SF_dine_percentage['Population'].replace({',':''},regex=True).apply(pd.to_numeric,1)


In [31]:
# Drop columns that will not be analyzed in this step
SF_dine_percentage=SF_dine_percentage.drop(['Zip Code','latitude','longitude'], 1)

In [32]:
# Create the normalizer data series 
divider=SF_dine_percentage['Population']

In [33]:
# To perform division on the entire dataframe, we need to create a temporary dataset and drop the 'Neighborhood' column to make all data numeric
temp_df=SF_dine_percentage
temp_df=temp_df.drop('Neighborhood',1)

In [34]:
temp_df=temp_df*10000

In [35]:
temp_df=temp_df.divide(divider, 0)

In [36]:
#Add back the 'Neighborhood' column
temp_df=pd.concat([temp_df, SF_dine_percentage['Neighborhood']], axis=1)

In [38]:
# Move neighborhood column to the first column
fixed_columns_2 = [temp_df.columns[-1]] + list(temp_df.columns[:-1])
SF_dine_final = temp_df[fixed_columns_2]
SF_dine_final = SF_dine_final.drop(['Population'],1)
SF_dine_final.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Burmese Restaurant,Caribbean Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Greek Restaurant,Hawaiian Restaurant,Hunan Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,North Indian Restaurant,Persian Restaurant,Peruvian Restaurant,Ramen Restaurant,Restaurant,Salvadoran Restaurant,Seafood Restaurant,Sicilian Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Bar,Beer Bar,Cocktail Bar,Dive Bar,Gay Bar,Hotel Bar,Juice Bar,Karaoke Bar,Sake Bar,Salon / Barbershop,Sports Bar,Tiki Bar,Wine Bar,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Street Food Gathering
0,Bayview-Hunters Point,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,Castro/Noe Valley,0.0,0.003941,0.0,0.000000,0.003941,0.00000,0.007881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.003941,0.000000,0.0,0.000000,0.003941,0.0,0.003941,0.000000,0.0,0.003941,0.000000,0.015763,0.00000,0.0,0.000000,0.0,0.003941,0.000000,0.003941,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.011822,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.003941,0.000000,0.000000,0.0,0.00000,0.000000,0.003941,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,Chinatown,0.0,0.016955,0.0,0.008478,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008478,0.000000,0.000000,0.008478,0.0,0.000000,0.016955,0.0,0.000000,0.008478,0.0,0.000000,0.008478,0.008478,0.00000,0.0,0.008478,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.016955,0.0,0.00000,0.000000,0.0,0.000000,0.008478,0.016955,0.008478,0.033910,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.008478,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,Haight-Ashbury,0.0,0.003110,0.0,0.000000,0.000000,0.00311,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.003110,0.003110,0.0,0.000000,0.

#### Print each neighborhood along with the top 5 most common venues

In [39]:
num_top_venues = 5

for hood in SF_dine_final['Neighborhood']:
    print("----"+hood+"----")
    temp = SF_dine_final[SF_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bayview-Hunters Point----
               venue  freq
0  Afghan Restaurant   0.0
1         Sports Bar   0.0
2           Sake Bar   0.0
3        Karaoke Bar   0.0
4          Juice Bar   0.0


----Castro/Noe Valley----
                venue  freq
0  Mexican Restaurant  0.02
1    Sushi Restaurant  0.01
2  Chinese Restaurant  0.01
3   Afghan Restaurant  0.00
4            Beer Bar  0.00


----Chinatown----
                 venue  freq
0         Cocktail Bar  0.03
1     Sushi Restaurant  0.02
2   Italian Restaurant  0.02
3  American Restaurant  0.02
4                  Bar  0.02


----Haight-Ashbury----
               venue  freq
0  Afghan Restaurant   0.0
1         Sports Bar   0.0
2           Sake Bar   0.0
3        Karaoke Bar   0.0
4          Juice Bar   0.0


----Hayes Valley/Tenderloin/North of Market----
                             venue  freq
0  Southern / Soul Food Restaurant  0.01
1                French Restaurant  0.01
2  Southern / Soul Food Restaurant  0.01
3  Southern / Sou

Write a function to sort the venues in descending order.

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = SF_dine_final['Neighborhood']

for ind in np.arange(SF_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(SF_dine_final.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bayview-Hunters Point,Street Food Gathering,Indian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant
1,Castro/Noe Valley,Mexican Restaurant,Sushi Restaurant,Chinese Restaurant,Kebab Restaurant,American Restaurant,Italian Restaurant,Burmese Restaurant,Malay Restaurant,Restaurant,Greek Restaurant
2,Chinatown,Cocktail Bar,American Restaurant,Italian Restaurant,Sushi Restaurant,Bar,Korean Restaurant,Vietnamese Restaurant,Asian Restaurant,Salon / Barbershop,Hawaiian Restaurant
3,Haight-Ashbury,Indian Restaurant,Mexican Restaurant,Cocktail Bar,Sports Bar,Bar,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Tapas Restaurant
4,Hayes Valley/Tenderloin/North of Market,Vietnamese Restaurant,Restaurant,Vegetarian / Vegan Restaurant,French Restaurant,Wine Bar,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Thai Restaurant,Sushi Restaurant,Southern / Soul Food Restaurant
5,Ingelside-Excelsior/Crocker-Amazon,Mexican Restaurant,Vietnamese Restaurant,Bar,Fast Food Restaurant,Fast Food Restaurant,Japanese Restaurant,Latin American Restaurant,Hunan Restaurant,Filipino Restaurant,Fast Food Restaurant
6,Inner Mission/Bernal Heights,Mexican Restaurant,New American Restaurant,Fast Food Restaurant,Fast Food Restaurant,Indian Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
7,Inner Richmond,Sushi Restaurant,Bar,Vietnamese Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Burmese Restaurant,Southern / Soul Food Restaurant,Thai Restaurant,Southern / Soul Food Restaurant
8,Lake Merced,Juice Bar,Cocktail Bar,Food Truck,Mexican Restaurant,Street Food Gathering,Middle Eastern Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant
9,Marina,Italian Restaurant,Vegetarian / Vegan Restaurant,American Restaurant,Caribbean Restaurant,Mediterranean Restaurant,Spanish Restaurant,Sushi Restaurant,Mexican Restaurant,Bar,Kebab Restaurant


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into several clusters.
Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [53]:
# set number of clusters
kclusters = 8

SF_grouped_clustering = SF_dine_final.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(SF_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

SF_merged = df_SFdata

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
SF_merged = SF_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [54]:
SF_merged

,Zip Code,Neighborhood,latitude,longitude,Population,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,94102,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,"28,991",6,Vietnamese Restaurant,Restaurant,Vegetarian / Vegan Restaurant,French Restaurant,Wine Bar,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Thai Restaurant,Sushi Restaurant,Southern / Soul Food Restaurant
1,94103,South of Market,37.77,-122.41,"23,016",7,Gay Bar,American Restaurant,Cocktail Bar,Thai Restaurant,Food Truck,Street Food Gathering,Middle Eastern Restaurant,Japanese Restaurant,Bar,Sports Bar
2,94107,Potrero Hill,37.76,-122.39,"17,368",2,Food Truck,Wine Bar,Cocktail Bar,Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Italian Restaurant,Sushi Restaurant,Latin American Restaurant,Southern / Soul Food Restaurant
3,94108,Chinatown,37.79,-122.41,"13,716",5,Cocktail Bar,American Restaurant,Italian Restaurant,Sushi Restaurant,Bar,Korean Restaurant,Vietnamese Restaurant,Asian Restaurant,Salon / Barbershop,Hawaiian Restaurant
4,94109,Polk/Russian Hill (Nob Hill),37.79,-122.42,"56,322",1,Sushi Restaurant,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Bar,Cocktail Bar,American Restaurant,Seafood Restaurant,Mexican Restaurant,Afghan Restaurant
5,94110,Inner Mission/Bernal Heights,37.74,-122.41,"74,633",1,Mexican Restaurant,New American Restaurant,Fast Food Restaurant,Fast Food Restaurant,Indian Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
6,94112,Ingelside-Excelsior/Crocker-Amazon,37.72,-122.44,"73,104",1,Mexican Restaurant,Vietnamese Restaurant,Bar,Fast Food Restaurant,Fast Food Restaurant,Japanese Restaurant,Latin American Restaurant,Hunan Restaurant,Filipino Restaurant,Fast Food Restaurant
7,94114,Castro/Noe Valley,37.75,-122.43,"30,574",1,Mexican Restaurant,Sushi Restaurant,Chinese Restaurant,Kebab Restaurant,American Restaurant,Italian Restaurant,Burmese Restaurant,Malay Restaurant,Restaurant,Greek Restaurant
8,94115,Western Addition/Japantown,37.78,-122.43,"33,115",1,Karaoke Bar,New American Restaurant,American Restaurant,Seafood Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant
9,94116,Parkside/Forest Hill,37.74,-122.48,"42,958",3,Chinese Restaurant,Sushi Restaurant,Thai Restaurant,American Restaurant,Dive Bar,Cocktail Bar,Dim Sum Restaurant,Food Truck,Vietnamese Restaurant,Japanese Restaurant


Visualize the resulting clusters

In [55]:
# create map
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(SF_merged['latitude'], SF_merged['longitude'], SF_merged['Neighborhood'], SF_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
SF_merged.loc[SF_merged['Cluster Labels'] == 0, SF_merged.columns[[1] + list(range(5, SF_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Twin Peaks-Glen Park,0,Salon / Barbershop,Korean Restaurant,Dim Sum Restaurant,North Indian Restaurant,Japanese Restaurant,Kebab Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant


In [58]:
SF_merged.loc[SF_merged['Cluster Labels'] == 1, SF_merged.columns[[1] + list(range(5, SF_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Polk/Russian Hill (Nob Hill),1,Sushi Restaurant,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Bar,Cocktail Bar,American Restaurant,Seafood Restaurant,Mexican Restaurant,Afghan Restaurant
5,Inner Mission/Bernal Heights,1,Mexican Restaurant,New American Restaurant,Fast Food Restaurant,Fast Food Restaurant,Indian Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
6,Ingelside-Excelsior/Crocker-Amazon,1,Mexican Restaurant,Vietnamese Restaurant,Bar,Fast Food Restaurant,Fast Food Restaurant,Japanese Restaurant,Latin American Restaurant,Hunan Restaurant,Filipino Restaurant,Fast Food Restaurant
7,Castro/Noe Valley,1,Mexican Restaurant,Sushi Restaurant,Chinese Restaurant,Kebab Restaurant,American Restaurant,Italian Restaurant,Burmese Restaurant,Malay Restaurant,Restaurant,Greek Restaurant
8,Western Addition/Japantown,1,Karaoke Bar,New American Restaurant,American Restaurant,Seafood Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant
10,Haight-Ashbury,1,Indian Restaurant,Mexican Restaurant,Cocktail Bar,Sports Bar,Bar,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Tapas Restaurant
12,Outer Richmond,1,Food Truck,Street Food Gathering,North Indian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant
13,Sunset,1,Vietnamese Restaurant,Szechuan Restaurant,Chinese Restaurant,Dim Sum Restaurant,Thai Restaurant,Dumpling Restaurant,Indian Restaurant,Salon / Barbershop,Bar,Middle Eastern Restaurant
15,Bayview-Hunters Point,1,Street Food Gathering,Indian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant
18,Lake Merced,1,Juice Bar,Cocktail Bar,Food Truck,Mexican Restaurant,Street Food Gathering,Middle Eastern Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant


In [47]:
SF_merged.loc[SF_merged['Cluster Labels'] == 2, SF_merged.columns[[1] + list(range(5, SF_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Potrero Hill,2,Food Truck,Wine Bar,Cocktail Bar,Restaurant,Southern / Soul Food Restaurant,Southern / Soul Food Restaurant,Italian Restaurant,Sushi Restaurant,Latin American Restaurant,Southern / Soul Food Restaurant


In [48]:
SF_merged.loc[SF_merged['Cluster Labels'] == 3, SF_merged.columns[[1] + list(range(5, SF_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,South of Market,3,Gay Bar,American Restaurant,Cocktail Bar,Thai Restaurant,Food Truck,Street Food Gathering,Middle Eastern Restaurant,Japanese Restaurant,Bar,Sports Bar
4,Polk/Russian Hill (Nob Hill),3,Sushi Restaurant,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Bar,Cocktail Bar,American Restaurant,Seafood Restaurant,Mexican Restaurant,Afghan Restaurant
5,Inner Mission/Bernal Heights,3,Mexican Restaurant,New American Restaurant,Fast Food Restaurant,Fast Food Restaurant,Indian Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
6,Ingelside-Excelsior/Crocker-Amazon,3,Mexican Restaurant,Vietnamese Restaurant,Bar,Fast Food Restaurant,Fast Food Restaurant,Japanese Restaurant,Latin American Restaurant,Hunan Restaurant,Filipino Restaurant,Fast Food Restaurant
7,Castro/Noe Valley,3,Mexican Restaurant,Sushi Restaurant,Chinese Restaurant,Kebab Restaurant,American Restaurant,Italian Restaurant,Burmese Restaurant,Malay Restaurant,Restaurant,Greek Restaurant
8,Western Addition/Japantown,3,Karaoke Bar,New American Restaurant,American Restaurant,Seafood Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant
10,Haight-Ashbury,3,Indian Restaurant,Mexican Restaurant,Cocktail Bar,Sports Bar,Bar,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Tapas Restaurant
12,Outer Richmond,3,Food Truck,Street Food Gathering,North Indian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant
13,Sunset,3,Vietnamese Restaurant,Szechuan Restaurant,Chinese Restaurant,Dim Sum Restaurant,Thai Restaurant,Dumpling Restaurant,Indian Restaurant,Salon / Barbershop,Bar,Middle Eastern Restaurant
14,Marina,3,Italian Restaurant,Vegetarian / Vegan Restaurant,American Restaurant,Caribbean Restaurant,Mediterranean Restaurant,Spanish Restaurant,Sushi Restaurant,Mexican Restaurant,Bar,Kebab Restaurant


In [49]:
SF_merged.loc[SF_merged['Cluster Labels'] == 4, SF_merged.columns[[1] + list(range(5, SF_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,St. Francis Wood/Miraloma/West Portal,4,Bar,Thai Restaurant,Street Food Gathering,New American Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant
